# 02_前処理大全_コード演習_第4章_結合

## 環境設定

In [1]:
#tabキー入力補完
%config IPCompleter.greedy=True

In [2]:
#データアクセス操作用ライブラリ
import os
import re 
from pathlib import Path
import csv
import pickle

#演算処理・加工集計用ライブラリ
import numpy as np
import pandas as pd

#可視化用ライブラリ
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

#表示設定
##アウトプット行に図を出力
%matplotlib inline
##出力する図を高解像度化
%config InlineBackend.figure_formats = {'png', 'retina'}
##明度のばらつきの少ないhuslを指定
sns.set_palette('husl')
##表示最大行数を100に設定
pd.options.display.max_rows = 100
##表示最大列数を無限に設定
pd.options.display.max_columns = None
##表示桁数を小数点以下2桁までに設定
#pd.options.display.float_format = '{:,2f}'.format

##不要な警告を非表示にする
import warnings
warnings.filterwarnings('ignore')

In [3]:
#現在のディレクトリを確認
os.getcwd()

'/Users/harukatsuno/Documents/notebook'

## データ読み込み

In [4]:
HOME = Path.cwd()
DATASET = HOME.parent/'dataset'/'awesome_hotel'
customer_file  = './customer.csv'
hotel_file = './hotel.csv'
reserve_file = './reserve.csv'

In [5]:
df_customer = pd.read_csv(DATASET/customer_file, encoding='utf-8')
df_customer.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


In [6]:
df_hotel = pd.read_csv(DATASET/hotel_file, encoding='utf-8')
df_hotel.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


In [7]:
df_reserve = pd.read_csv(DATASET/reserve_file, encoding='utf-8')
df_reserve.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


## マスタテーブルの結合

In [8]:
#予約テーブルとホテルテーブルを結合して、宿泊人数が1のビジネスホテルの予約レコードにも抽出する。
pd.merge(df_reserve.query('people_num == 1'), df_hotel.query('is_business == True'), on='hotel_id', how='left')

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500.0,C,C-1,38.237294,140.696131,True
1,r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,NaN,NaN,NaN,NaN,NaN,NaN
2,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900.0,G,G-4,33.595248,130.633567,True
3,r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500.0,C,C-2,38.329097,140.698165,True
4,r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400.0,C,C-1,38.231842,140.797268,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,r4005,h_150,c_994,2016-03-25 11:51:26,2016-04-02,09:00:00,2016-04-03,1,25200,25200.0,B,B-2,35.546229,139.693687,True
965,r4011,h_137,c_995,2017-08-13 19:23:48,2017-09-07,09:30:00,2017-09-09,1,12400,6200.0,F,F-1,34.538252,132.469507,True
966,r4012,h_54,c_995,2018-03-08 03:54:07,2018-03-20,10:30:00,2018-03-22,1,25600,NaN,NaN,NaN,NaN,NaN,NaN
967,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1,37200,NaN,NaN,NaN,NaN,NaN,NaN


# ホテルごとに他のホテルをレコメンド

In [13]:
#ホテルのレコメンド候補が小地域(small_area_name)が20件以上ある場合は、同じ小地域のホテルをレコメンドする。
#小地域のレコメンド候補が20件に満たない場合は同じ大地域(big_area_name)のホテルをレコメンドする。
#ガベージコレクション（必要ないメモリの開放）のためのライブラリ


#small_area_nameごとにホテル数をカウント※列名をindex にしたく無い場合は as_index=False 
small_area_mst = df_hotel.groupby(['big_area_name', 'small_area_name'], as_index=False).size()
small_area_mst.columns = ['big_area_name', 'small_area_name', 'hotel_cnt']
small_area_mst.head()

#20件以上であればjoin_area_idをsmall_area_nameとして設定
#20 件未満であればjoin_area_idをbig_area_nameとして設定
#-1は自ホテルを引く
small_area_mst['join_area_id'] = np.where(small_area_mst['hotel_cnt'] -1 >= 20,
                                                                            small_area_mst['small_area_name'],
                                                                            small_area_mst['big_area_name'])

#必要なくなった列を削除
small_area_mst.drop(['hotel_cnt', 'big_area_name'], axis=1, inplace=True)

#レコメンドもとになるホテルにsmall_area_mstを結合することで、join_area_idを設定
base_hotel_mst = pd.merge(df_hotel, small_area_mst, on='small_area_name', how='left').loc[:, ['hotel_id', 'join_area_id']]

#下記は必要に応じてメモリを開放(必須ではないがメモリ量に余裕がない時に利用)
del small_area_mst
gc.collect()

#recomend_hotel_mstはレコメンド候補のためのテーブル
recomend_hotel_mst = pd.concat([df_hotel[['small_area_name', '']]])#join_area_idを　big_areaとしたレコメンド候補マスタ

base_hotel_mst.head()

,hotel_id,join_area_id
0,h_1,D
1,h_2,A-1
2,h_3,E
3,h_4,C-3
4,h_5,G


In [10]:
#各ホテルの予約金額の分散値と標準偏差値を算出　※ただし、予約が1件しかない場合は分散値と標準偏差値を0とする
result_d = df_reserve.groupby('hotel_id').agg(
    {'total_price':['var', 'std']}).reset_index()
result_d.columns = ['hotel_id', 'price_var', 'price_std']
result_d.fillna(0, inplace=True)
result_d.head()

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159


In [52]:
#3桁以降の金額を丸めた予約金額をカテゴリ化して最頻値を算出
df_reserve['total_price'].round(-3).mode()

0    10000
1    20000
2    40000
dtype: int64

In [59]:
#顧客ごとに予約日時の順位を古い順に付与　※同じ日付の場合データの読み込み順に順位が小さくなるように指定する
df_reserve['reserve_datetime'] = pd.to_datetime(
    df_reserve['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')
df_reserve['rank_no'] = df_reserve.groupby('customer_id')['reserve_datetime'].rank(ascending=True, method='first')
df_reserve.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,rank_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5.0


In [70]:
#ホテルごとの予約数の順位を算出　※同じ予約数の場合最小となる順位となうように指定する
df_reserve_rank = df_reserve.groupby('hotel_id').size().reset_index()
df_reserve_rank.columns = ['hotel_id', 'rsv_cnt']
df_reserve_rank['rsv_cnt_rank'] = df_reserve_rank['rsv_cnt'].rank(ascending=True, method='min')
df_reserve_rank.drop('rsv_cnt', axis=1, inplace=True)
df_reserve_rank.head()

,hotel_id,rsv_cnt_rank
0,h_1,42.0
1,h_10,1.0
2,h_100,282.0
3,h_101,242.0
4,h_102,127.0
